<a href="https://colab.research.google.com/github/SantiagoGomezfpv/hyperparameter/blob/main/pytorchModel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
pip install optuna

In [7]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from optuna.trial import Trial
from optuna.pruning import MedianPruner
from optuna.distributions import UniformDistribution, CategoricalDistribution

ModuleNotFoundError: No module named 'OptunaPruningExtension'

In [ ]:
# Transformaciones para normalizar las imágenes
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5), (0.5))
])

# Carga de los conjuntos de datos de entrenamiento y prueba
train_dataset = datasets.MNIST(root='./data', train=True, download=True, transform=transform)
test_dataset = datasets.MNIST(root='./data', train=False, transform=transform)

# Creación de cargadores de datos
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=64, shuffle=False)
